In [1]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

from surprise import Dataset, Reader, accuracy , KNNWithMeans
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from surprise.prediction_algorithms import SVD, SVDpp, NormalPredictor , BaselineOnly

plt.style.use('fast')
%matplotlib inline

In [2]:
music_review2=pd.read_csv('./Data/music_review.csv')

In [3]:
music_review2

,Unnamed: 0,overall,reviewerID,asin,reviewText
0,0,5,A1ZCPG3D3HGRSS,0001388703,This is a great cd full of worship favorites!!...
1,1,5,AC2PL52NKPL29,0001388703,"So creative! Love his music - the words, the ..."
2,2,5,A1SUZXBDZSDQ3A,0001388703,"Keith Green, gone far to early in his carreer,..."
3,3,5,A3A0W7FZXM0IZW,0001388703,Keith Green had his special comedy style of Ch...
4,4,5,A12R54MKO17TW0,0001388703,Keith Green / So you wanna go back to Egypt......
...,...,...,...,...,...
1517681,1584077,5,AR3KABMPL5L0O,B01HJ91P94,Casting Crowns....you do it so well! Awesome s...
1517682,1584078,4,A2N53GHW73INDH,B01HJ91P94,This band has produced many inspiring Christia...
1517683,1584079,5,ABNKLDCCVJKW1,B01HJ91P94,Awesome band and awesome song. This is my next...
1517684,1584080,5,AMWSDABZWFRAT,B01HJ91IVY,Excellent


In [4]:
# Convert 'reviewText' column to strings
music_review2['reviewText'] = music_review2['reviewText'].astype(str)

# Group by 'asin' and join review text
grouped_reviews = music_review2.groupby('asin')['reviewText'].agg(lambda x: ' '.join(x))

# Convert the result back to a DataFrame
grouped_reviews_df = pd.DataFrame(grouped_reviews).reset_index()


In [5]:
grouped_reviews_df

,asin,reviewText
0,0001377647,"If you're looking for a meditative, contemplat..."
1,0001388703,This is a great cd full of worship favorites!!...
2,0001526146,"This is music from my younger years that I, as..."
3,0001527134,"Don Francisco's ""Early Works"" are filled with ..."
4,0001529145,"Discovering older Christian music, inspiration..."
...,...,...
456806,B01HJ91RWE,Love this group!
456807,B01HJ91TDQ,"This was the song as I've heard it on T.V., do..."
456808,B01HJ91VJ8,"This is a beautiful, worshipful song that glor..."
456809,B01HJ91WOW,"Awesome Love, love, love it Love It,Anytime..."


In [8]:
grouped_reviews_df.shape

(456811, 2)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize the review text using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(grouped_reviews_df['reviewText'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Concatenate the 'asin' column with the TF-IDF DataFrame
tfidf_df = pd.concat([grouped_reviews_df['asin'], tfidf_df], axis=1)

MemoryError: Unable to allocate 1.04 TiB for an array with shape (456811, 313155) and data type float64